In [7]:
import requests
from bs4 import BeautifulSoup
import csv
import math


In [8]:
# Danh sách chuyên mục và đường dẫn
topics = [
    {"name": "Thời sự", "tag": "thoi-su"},
    {"name": "Thế giới", "tag": "the-gioi"},
    {"name": "Kinh doanh", "tag": "kinh-doanh"},
    {"name": "Bất động sản", "tag": "bat-dong-san"},
    {"name": "Khoa học", "tag": "khoa-hoc"},
    {"name": "Giải trí", "tag": "giai-tri"},
    {"name": "Thể thao", "tag": "the-thao"},
    {"name": "Pháp luật", "tag": "phap-luat"},
    {"name": "Giáo dục", "tag": "giao-duc"},
    {"name": "Sức khỏe", "tag": "suc-khoe"},
    {"name": "Đời sống", "tag": "doi-song"},
    {"name": "Du lịch", "tag": "du-lich"},
    {"name": "Số hóa", "tag": "so-hoa"},
    {"name": "Ôtô - Xe máy", "tag": "oto-xe-may"},
    {"name": "Ý kiến", "tag": "y-kien"},
]



In [9]:
# Số lượng bài viết cần crawl
TOTAL_ARTICLES = 10000

In [10]:
# Base URL của trang VnExpress
base_url = "https://vnexpress.net/"

# Tính số lượng bài viết tối đa cho mỗi chuyên mục
articles_per_topic = math.ceil(TOTAL_ARTICLES / len(topics))
print(f"Số lượng bài viết cần crawl cho mỗi chuyên mục: {articles_per_topic}")

Số lượng bài viết cần crawl cho mỗi chuyên mục: 667


In [11]:
def crawl_articles(base_url, topic, max_titles):
    """Crawl bài báo từ một chuyên mục."""
    topic_name = topic["name"]
    tag = topic["tag"]
    titles = []
    count_titles = 0
    page = 1

    while count_titles < max_titles:
        # Xây dựng URL phân trang
        if page == 1:
            url = f"{base_url}{tag}"
        else:
            url = f"{base_url}{tag}-p{page}"
        
        # Gửi request đến URL
        response = requests.get(url)
        if response.status_code != 200:
            print(f"Không thể truy cập {url}. Mã lỗi: {response.status_code}")
            break
        
        # Parse HTML để lấy bài báo
        soup = BeautifulSoup(response.content, "html.parser")
        articles = soup.select(".title-news a[title]")
        if not articles:  # Nếu không còn bài viết, thoát vòng lặp
            break
        
        # Lấy tiêu đề các bài báo
        for article in articles:
            if count_titles >= max_titles:
                break
            title = article.get("title").strip()
            titles.append((title, topic_name))
            count_titles += 1
        
        page += 1
    
    return titles

In [12]:
def save_to_csv(data, filename="vnexpress_articles.csv"):
    """Ghi dữ liệu vào file CSV."""
    with open(filename, mode="w", encoding="utf-8", newline="") as file:
        writer = csv.writer(file)
        writer.writerow(["Title", "Topic"])  # Header
        writer.writerows(data)
    print(f"Đã lưu {len(data)} bài báo vào file '{filename}'")

In [13]:
# Crawl dữ liệu
all_articles = []

for topic in topics:
    print(f"Đang crawl chuyên mục: {topic['name']}...")
    articles = crawl_articles(base_url, topic, articles_per_topic)
    all_articles.extend(articles)

print(f"Tổng số bài báo đã crawl: {len(all_articles)}")

Đang crawl chuyên mục: Thời sự...
Đang crawl chuyên mục: Thế giới...
Đang crawl chuyên mục: Kinh doanh...
Đang crawl chuyên mục: Bất động sản...
Đang crawl chuyên mục: Khoa học...
Đang crawl chuyên mục: Giải trí...
Đang crawl chuyên mục: Thể thao...
Đang crawl chuyên mục: Pháp luật...
Đang crawl chuyên mục: Giáo dục...
Đang crawl chuyên mục: Sức khỏe...
Đang crawl chuyên mục: Đời sống...
Đang crawl chuyên mục: Du lịch...
Đang crawl chuyên mục: Số hóa...
Đang crawl chuyên mục: Ôtô - Xe máy...
Đang crawl chuyên mục: Ý kiến...
Tổng số bài báo đã crawl: 9293


In [14]:
# Ghi dữ liệu vào file CSV
save_to_csv(all_articles, "vnexpress_articles.csv")


Đã lưu 9293 bài báo vào file 'vnexpress_articles.csv'


In [15]:
import pandas as pd

# Đọc file CSV để kiểm tra
data = pd.read_csv("vnexpress_articles.csv")
data.head(10)


,Title,Topic
0,Dự kiến tên các Bộ sau hợp nhất,Thời sự
1,Cụm dân cư gần 400 người 'không được thừa nhận...,Thời sự
2,Ông Lê Doãn Hợp: '9-10 người dân nuôi một ngườ...,Thời sự
3,TP HCM dự tính bắn pháo hoa 3 điểm dịp Tết Dươ...,Thời sự
4,Dự kiến bắn 21 loạt đại bác kỷ niệm 50 năm thố...,Thời sự
5,Tổng Bí thư: Chọn công nghệ tốt nhất làm điện ...,Thời sự
6,Đề nghị cấp bằng Tổ quốc ghi công cho 12 quân ...,Thời sự
7,Thủ tướng: Đa dạng nguồn lực để xây đường sắt ...,Thời sự
8,Nam công nhân lao mình cứu người phụ nữ nhảy cầu,Thời sự
9,Nguy cơ tai nạn từ khách nước ngoài chạy xe má...,Thời sự
